### 1. Import packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### 2. Read the 3 datasets into dataframes

In [ ]:
data = pd.read_csv("Data.csv")
item = pd.read_csv("item.csv")
user = pd.read_csv("user.csv")

### 3. Apply info, shape, describe, and find the number of missing values in the data.

#### Applying functions to the Data.csv file

In [ ]:
data.info()
print("\nThe data shape is:", data.shape)
data.describe()

In [ ]:
data.isnull().values.any()

#### Applying functions to the item.csv file

In [ ]:
item.info()
print("\nThe data shape is:", item.shape)
item.describe()

In [ ]:
item.isnull().values.any()

#### Applying functions to the user.csv file

In [ ]:
user.info()
print("\nThe data shape is:", user.shape)
user.describe()

In [ ]:
user.isnull().values.any()

**Observations:**  
1. 1682 different movies were given ratings concluded from the movie id max and the shape of item.csv.  
2. 943 users reviewed movies which was concluded from the user id max from Data.csv and the shape of user.csv.  
3. The average movie rating across all 100000 ratings is 3.52986.  
4. There are no missing values in Data.csv, item.csv, or user.csv.


### 4. Find the number of movies per genre using the item data

In [ ]:
item_sum_df = pd.DataFrame(item.sum(), columns = ["Count"])
drop = ["movie id", "movie title", "release date"]
genres = item_sum_df.drop(index = drop, axis = 0).sort_values("Count", ascending = False)
genres

**Insights:**
1. There is 1 movie that is defined with the genre 'unknown'.
2. The 'Drama' genre has the most ratings with 725.
3. The 'Fantasy' genre has the least ratings with 22.

### 5. Drop the movie where the genre is unknown

In [ ]:
item_df = pd.DataFrame(item)
removed_movie_title = item_df["movie title"][item_df.index[item_df['unknown'] == 1][0]]
removed_movie_release_date = item_df["release date"][item_df.index[item_df['unknown'] == 1][0]]
item_df_1 = item_df.drop(index = item_df.index[item_df['unknown'] == 1], inplace = False)
print("The shape of the original item df is", item_df.shape)
print("The shape of the new item df without the movie listed as 'unkown' is", item_df_1.shape)
print("The name of the movie removed is", removed_movie_title, "which was released on", removed_movie_release_date)

**Insights:**
1. The shape of the original item df is (1681, 22)
2. The shape of the new item df without the movie listed as 'unkown' is (1680, 22)
3. The name of the movie removed is Good Morning  which was released on 4-Feb-1971



### 6. Find the movies that have more than one genre

#### Dataframe which displays the movie name and its number of genres.


In [ ]:
num_of_genres = pd.DataFrame(item_df_1.iloc[:, 4:].sum(axis = 1), columns = ["Number of Generes"])
title_genres = pd.concat([item_df_1["movie title"], num_of_genres], axis = 1)
title_genres

#### Determining the number of movies which have more than one genre.


In [ ]:
#Add movie genres wihtin row with the below code
#title_genres.sum(axis = 1)

#Determine which movies have more than 1 genre category with the below code and output NaN if == 1.
#title_genres.sum(axis = 1).where(title_genres["Number of Generes"] > 1)

#Drop andy Nan values and display array length
print("The number of movies which have more than one genre listed is"
      , len(title_genres.sum(axis = 1).where(title_genres["Number of Generes"] > 1).dropna()))

**Insights:**
1. There are 1680 movies with valid listed movie genres (genre is not 'unkown') 
2. There are 849 movies with more than one genre listed
3. There are 831 movies with only one genre listed



### 7. Univariate plots of columns: 'rating', 'Age', 'release year', 'Gender' and 'Occupation'

In [ ]:
a = 'My*cat*is*brown'
print(a.split('*'))

#similarly, the release year needs to be taken out from release date

#also you can simply slice existing string to get the desired data, if we want to take out the colour of the cat

print(a[10:])
print(a[-5:])

#### Displot for Age

In [ ]:
sns.displot(user["age"], kde = True);

#### Countplot for Ratings

In [ ]:
sns.countplot(x = "rating", data = data);

#### Countplot for Gender

In [ ]:
sns.countplot(x = "gender", data = user);

#### Countplot for Occupation

In [ ]:
fig, ax = plt.subplots(figsize = (15, 15))
ax.grid()
sns.countplot(y = "occupation", data = user, order = user['occupation'].value_counts().index, ax = ax);

#### Countplot for release year

In [ ]:
import re
text_to_search = " ".join(item_df_1["release date"])
pattern = re.compile(r'\d{4}\b')
matches = pattern.finditer(text_to_search)
c = []
for mat in matches:
    c.append(mat.group(0))

In [ ]:
fig, ax = plt.subplots(figsize = (15, 15))
ax.grid()
release_year = pd.DataFrame(c, columns = ["release year"]).sort_values(by = "release year", ascending = False)
sns.countplot(y = "release year", data = release_year); 

### 8. Visualize how popularity of genres has changed over the years

#### Below is a data frame where the release year is the index and the genre is the column names

In [ ]:
#first made an array of all the years each movie was released in 
date = []
for i in item_df_1["release date"]:
    date.append(int(i[-4:]))

#converted array into DF
#joined DF with item_df_1 and dropped the columns ['movie id','movie title',"unknown","release date"]
#grouped by "release year" so there were not anymultiple rows with the same year
#added the movie genres released that year
#sorted by release year

dg_release_year = pd.DataFrame(date, columns = ["release year"])
                .join(item_df_1.drop(['movie id', 'movie title', "unknown", "release date"], 1))
                .sort_values(by = "release year")
dg = dg_release_year.groupby("release year")[list(dg_release_year.columns[1:])].sum()
dg

#### Below is the total number of movie genres released in a year as a percentage

In [ ]:
# item dataset to get count of movies released in a particular year, store that value in a new column as 'total'
# divide the value of each genre in that year by total percentage number of release in a particular year. `
#(df.div(df['total'], axis = 0)*100)
year_total = pd.DataFrame(dg.sum(axis = 1), columns = ["total percent"])
year_total

dg_1 = dg.join(year_total).div(dg.join(year_total)["total percent"], axis = 0)
fig, ax = plt.subplots(figsize = (20, 20))
sns.heatmap(data = dg_1, annot = True, fmt = '.0%');

**Insights:**
- Drama movies have been produced since 1922 to 1998.
- Comedy movies seem to be the second most produced through the years 1922 - 1998. 
- There appears to be a large quantity of Horror and Drama movies released in 1922, 1926, and 1930. However; there was only one movie released those years within this sample.


### 9. Find the top 25 movies in terms of average ratings for movies that have been rated more than 100 times

#### Find the count of ratings and average ratings for every movie

In [ ]:
movie_id_avg_rate = data[["movie id", "rating"]].groupby(by = "movie id").mean().round(decimals = 2)
movie_id_count = pd.DataFrame(data["movie id"].value_counts()).rename(columns = {"movie id": "count"})
movie_id_avg_rate_count = movie_id_avg_rate.join(movie_id_count, sort = True)

movie_id_avg_rate_count

#### Slice the movies which have more than 100 ratings and sort according to average rating such that movie which highest rating is on top.

In [ ]:
movie_id_count_100 = pd.DataFrame(movie_id_avg_rate_count[movie_id_avg_rate_count['count'] >= 100])
                    .sort_values(by = "rating", ascending = False)
movie_id_count_100

#### Select top 25 movies

In [ ]:
#The below section checks if the movie with an unknown genre description is in the top 25 list
top_25 = movie_id_count_100.head(25)
movie_unknown_genre = item_df.index[item_df['unknown'] == 1][0]
if item_df.index[item_df['unknown'] == 1][0] not in list(top_25.sort_index().index):
    print("NOTE: The name of the movie with an 'unknown' genre is Good Morning which has a movie id:"
          , movie_unknown_genre
          ,"is not in the top 25 movies.")
    
#The below section selects the top 25 movies and displays their name, avg rating, and rating count 

top_25_movies = pd.merge(top_25, item[["movie id","movie title"]].set_index(keys = "movie id"), how = 'inner', on = 'movie id')[["movie title","rating","count"]]
top_25_movies


### 10. Check for the validity of the below statements with respect to the data provided

* Men watch more drama than women
* Women watch more Sci-Fi than men
* Men watch more Romance than women

In [ ]:
ud = user.merge(data, on = "user id", how = "inner")
df = item.merge(ud, on = "movie id", how = "inner")
df

In [ ]:
genre_by_gender = df.groupby("gender").sum().loc[:, "Action":"Western"]
genre_by_gender

In [ ]:
genre_by_gender["Total"] = df["gender"].value_counts()
genre_by_gender

In [ ]:
genre_by_gender = genre_by_gender.div(genre_by_gender.Total, axis = 0)*100
genre_by_gender

In [ ]:
fig, ax = plt.subplots(ncols = 3, figsize = (18, 5))

genre_by_gender["Drama"].plot(kind = "barh", ax = ax[0], label = "Drama")
ax[0].legend();

genre_by_gender["Sci-Fi"].plot(kind = "barh", ax = ax[1], label = "Sci-Fi")
ax[1].legend();

genre_by_gender["Romance"].plot(kind = "barh", ax = ax[2], label = "Romance")
ax[2].legend();

plt.show()

In [ ]:
genre_by_gender[["Drama", "Sci-Fi", "Romance"]]

**Conclusion:**
* Men watch more Drama than women.
        False. 42.77% of total ratings by Women were for Drama movies compared to 38.90% of total Mens ratings.
* Women watch more Sci-Fi than men.
        False. 10.21% of total ratings by Women were for Sci-Fi movies compared to 13.60% of total Mens ratings.
* Men watch more Romance than women.
        False. 22.76% of total ratings by Women were for Romance movies compared to 18.32% of total Mens ratings.
    
